<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/BelEvSer_NLP_HW_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00


In [1]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 23.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-8471dca11b8f9d26/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('corpus'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
device = "cpu"

In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("blinoff/medical_qa_ru_data")

model = AutoModelForSeq2SeqLM.from_pretrained("blinoff/medical_qa_ru_data").to(device)

OSError: ignored

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sberquad")

model = AutoModelForSeq2SeqLM.from_pretrained("sberquad").to(device)

OSError: ignored

In [27]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [29]:
text = "When should I go to the doctor?"

paraphrase(text)

['When is the appropriate time to see a healthcare provider?',
 'At what point should I seek medical attention?',
 'When should I see a doctor?',
 "What's the appropriate waiting period before seeing a doctor?",
 'What is the ideal waiting time to see a physician?']

In [31]:
text = "What is the capital of the USA?"

paraphrase(text)

['What city serves as the capital of the United States?',
 'Which city is designated as the capital of the United States?',
 'Where is the capital of USA?',
 'In which city is the main office of USA?',
 'In which city is the main office of USA?.']

In [32]:
text = "When was Ilon Mask born?"

paraphrase(text)

["What is the date of Ilon Mask's birth?",
 'At what time was Ilon Mask born?',
 'When was Ilon Mask born?',
 'Can you tell me the exact moment Ilon Mask was born?',
 'In which year was Ilon Mask born?']

In [33]:
text = "What does she do?"

paraphrase(text)

['What is her job?',
 'How does she work her job?',
 'What are her responsibilities?',
 'In what way is she engaged?',
 'Her job?']

In [34]:
text = "Where can I have a cup of coffe?"

paraphrase(text)

['Where is the best place to grab a cup of coffee?',
 'What are some places where I can enjoy a cup of coffee?',
 'In which location can I find a place to drink coffee?',
 'What are some places where I can enjoy a cup of coffee? Request',
 'In which location can I find a place to drink coffee? Request']